# IIT Madras PayBuddy Case Study
## An analysis by Team 1

### Importing necessary header files

In [58]:
import numpy as np
import pandas as pd
from functools import reduce
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import seaborn as sns

### Exploratory data analysis and data cleaning

#### Fraud Update Features

This dataset contains a wide range of information. Some of the information requires additional understanding

In [59]:
fraud_features=pd.read_csv("data/fraud_update_features.csv", index_col=0)

In [60]:
print("The dimensions of the dataset are: ", fraud_features.shape)

The dimensions of the dataset are:  (872890, 21)


In [61]:
fraud_features.head()

,cust_id,fraud_update_features_1,fraud_update_features_2,fraud_update_features_3,fraud_update_features_4,fraud_update_features_5,fraud_update_features_6,fraud_update_features_7,fraud_update_features_8,fraud_update_features_9,...,fraud_update_features_11,fraud_update_features_12,fraud_update_features_13,fraud_update_features_14,fraud_update_features_15,fraud_update_features_16,fraud_update_features_17,fraud_update_features_18,fraud_update_features_19,fraud_update_features_20
0,10021085,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,2020-05-21 11:45:42,NaN,NaN,NaN,NaN,1
1,10061508,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,2021-01-04 12:51:59,NaN,NaN,NaN,NaN,1
2,10104649,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
3,10107248,0,0,1,NaN,NaN,0,0,0,0,...,0,1,0,1,NaN,NaN,NaN,NaN,NaN,0
4,10124427,0,0,0,NaN,NaN,0,0,0,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0


Number of NaN values of each column

In [62]:
fraud_features.isnull().sum()[fraud_features.isnull().sum()>0]

fraud_update_features_4     853650
fraud_update_features_5     719848
fraud_update_features_15    669472
fraud_update_features_16    747217
fraud_update_features_17    747217
fraud_update_features_18    853650
fraud_update_features_19    719848
dtype: int64

Clearly from the above data, certain features have a large amount of missing data (>50%), so extrapolating data for those features would not make a lot of sense. As a result, we choose to drop those features from our analysis.

Features 4, 5, 15, 16 are timeseries data features, so those perhaps can be somehow considered in the analysis

In [63]:
fraud=fraud_features.drop(["fraud_update_features_4","fraud_update_features_5","fraud_update_features_15","fraud_update_features_16","fraud_update_features_17","fraud_update_features_18","fraud_update_features_19"], axis=1)


#### Payment Cashout Fail

In [64]:
payment_cashout_fail=pd.read_csv("data/payment_cashout_fail_df.csv", index_col=0)

In [65]:
cashout_fail_cols=payment_cashout_fail.columns
cashout_fail_cols=cashout_fail_cols[1:]

In [66]:
cashout_fail_cols

Index(['payment_cashout_fail_1', 'payment_cashout_fail_2',
       'payment_cashout_fail_3', 'payment_cashout_fail_4',
       'payment_cashout_fail_5', 'payment_cashout_fail_6',
       'payment_cashout_fail_7', 'payment_cashout_fail_8',
       'payment_cashout_fail_9', 'payment_cashout_fail_10',
       'payment_cashout_fail_11', 'payment_cashout_fail_12',
       'payment_cashout_fail_13', 'payment_cashout_fail_14',
       'payment_cashout_fail_15', 'payment_cashout_fail_16',
       'payment_cashout_fail_17', 'payment_cashout_fail_18',
       'payment_cashout_fail_19', 'payment_cashout_fail_20',
       'payment_cashout_fail_21', 'payment_cashout_fail_22',
       'payment_cashout_fail_23', 'payment_cashout_fail_24',
       'payment_cashout_fail_25', 'payment_cashout_fail_26',
       'payment_cashout_fail_27', 'payment_cashout_fail_28',
       'payment_cashout_fail_29', 'payment_cashout_fail_30',
       'payment_cashout_fail_31', 'payment_cashout_fail_32',
       'payment_cashout_fail_33',

In [67]:
print("The dimensions of the dataset are: ", payment_cashout_fail.shape)

The dimensions of the dataset are:  (49978, 57)


In [68]:
payment_cashout_fail.head()

,cust_id,payment_cashout_fail_1,payment_cashout_fail_2,payment_cashout_fail_3,payment_cashout_fail_4,payment_cashout_fail_5,payment_cashout_fail_6,payment_cashout_fail_7,payment_cashout_fail_8,payment_cashout_fail_9,...,payment_cashout_fail_47,payment_cashout_fail_48,payment_cashout_fail_49,payment_cashout_fail_50,payment_cashout_fail_51,payment_cashout_fail_52,payment_cashout_fail_53,payment_cashout_fail_54,payment_cashout_fail_55,payment_cashout_fail_56
0,10000827,111.87,111.87,111.87,111.87,111.87,111.87,0.0,0.0,1,...,111.87,111.87,0.0,0.0,1,1,1,0,0,55.0
1,10001467,49.50,49.50,49.50,49.50,49.50,0.00,0.0,0.0,1,...,49.50,0.00,0.0,0.0,1,1,0,0,0,126.0
2,10005135,141.08,141.08,141.08,141.08,0.00,0.00,0.0,0.0,1,...,0.00,0.00,0.0,0.0,1,0,0,0,0,190.0
3,10006258,297.00,148.50,148.50,148.50,297.00,0.00,0.0,0.0,2,...,297.00,0.00,0.0,0.0,2,2,0,0,0,174.0
4,10006555,29.70,29.70,29.70,29.70,0.00,0.00,0.0,0.0,1,...,0.00,0.00,0.0,0.0,1,0,0,0,0,239.0


Number of NaN values of each column

In [69]:
payment_cashout_fail.isnull().sum()[payment_cashout_fail.isnull().sum()>0]

payment_cashout_fail_28    44581
payment_cashout_fail_42     6665
payment_cashout_fail_56     2807
dtype: int64

While there are a lot of missing values in this dataset, it is important to note that we don't want to drop any features. This is because we suspect that the number of defaulted transactions would be extremely important when it comes to the actual data.

**In this case, we assume that NaN corresponds to 0**

cash_fail is the name of the processed dataset. The column "num" corresponds to the number of failed transactions. The column "avg" corresponds to the average value of money attempted during the failed cashouts  

In [70]:
cash_fail= payment_cashout_fail
cash_fail.fillna(0)
cash_fail["num"]= (cash_fail != 0).sum(axis=1)
cash_fail["avg"]= cash_fail.sum(axis=1)/cash_fail["num"]


#### New Account Financial Instrument

In [71]:
acct_finan_instr=pd.read_csv("data/new_acct_fi.csv", index_col=0)

In [126]:
acct_fi_cols=acct_finan_instr.columns
acct_fi_cols=acct_fi_cols[1:]

In [72]:
print("The dimensions of the dataset are: ", acct_finan_instr.shape)

The dimensions of the dataset are:  (569104, 18)


In [73]:
acct_finan_instr.head()

,cust_id,new_acct_fi_1,new_acct_fi_2,new_acct_fi_3,new_acct_fi_4,new_acct_fi_5,new_acct_fi_6,new_acct_fi_7,new_acct_fi_8,new_acct_fi_9,new_acct_fi_10,new_acct_fi_11,new_acct_fi_12,new_acct_fi_13,new_acct_fi_14,new_acct_fi_15,new_acct_fi_16,new_acct_fi_17
0,10004244,266.817095,1683.639988,1164.479865,267.041667,1684.041667,1164.694444,1,1,3,0,3,0,0,0,0,0,3
1,1001247,2552.330579,2552.330579,2552.330579,2553.000000,2553.000000,2553.000000,1,1,1,0,1,0,0,0,0,0,1
2,1002860,2303.076759,2303.076759,2303.076759,2302.041667,2302.041667,2302.041667,1,1,1,0,1,0,0,0,0,0,1
3,10044071,1757.191863,1757.191863,1757.191863,1758.041667,1758.041667,1758.041667,0,0,1,0,1,0,0,0,0,0,1
4,10045512,1031.925706,1273.035694,1152.480700,1033.041667,1243.041667,1138.041667,1,1,2,0,2,0,0,0,0,0,2


Number of NaN values of each column


In [74]:
acct_finan_instr.isnull().sum()[acct_finan_instr.isnull().sum()>0]

Series([], dtype: int64)

There are no missing values in this dataset, so there is no further data processing required. We shall make a copy of this dataset and call it finan_intsr

In [75]:
finan_instr=acct_finan_instr


#### New Account Receiver Engagement

In [76]:
acct_receiver=pd.read_csv("data/new_acct_receiver.csv", index_col=0)

In [77]:
print("The dimensions of the dataset are: ", acct_receiver.shape)

The dimensions of the dataset are:  (771148, 81)


In [78]:
acct_receiver.head()

,cust_id,acct_receiver_1,acct_receiver_2,acct_receiver_3,acct_receiver_4,acct_receiver_5,acct_receiver_6,acct_receiver_7,acct_receiver_8,acct_receiver_9,...,acct_receiver_71,acct_receiver_72,acct_receiver_73,acct_receiver_74,acct_receiver_75,acct_receiver_76,acct_receiver_77,acct_receiver_78,acct_receiver_79,acct_receiver_80
0,10001463,4,3,1,0,0,5,3,1,0,...,1730.0,830.00,0.00,0.0,0.0,2,1,0,0,0
1,10002983,13,4,4,0,0,17,5,5,0,...,281.5,5.50,5.50,0.0,0.0,10,1,1,0,0
2,10006303,8,7,6,1,0,51,29,16,2,...,67.0,67.00,47.00,0.0,0.0,3,3,2,0,0
3,10006693,1,0,0,0,0,3,0,0,0,...,65.0,0.00,0.00,0.0,0.0,3,0,0,0,0
4,10015613,8,1,1,0,0,15,1,1,0,...,326.8,202.97,202.97,0.0,0.0,8,1,1,0,0


Number of NaN values of each column

In [79]:
acct_receiver.isnull().sum()[acct_receiver.isnull().sum()>0]

Series([], dtype: int64)

There are no missing values in this dataset, so there is no further data processing required. We shall make a copy of this dataset and call it receiver

In [80]:
receiver=acct_receiver


#### New Account Sender Engagement

In [81]:
acct_sender=pd.read_csv("data/new_acct_sender.csv", index_col=0)

In [82]:
print("The dimensions of the dataset are: ", acct_sender.shape)

The dimensions of the dataset are:  (841975, 112)


In [83]:
acct_sender.head()

,cust_id,accnt_sender_1,accnt_sender_2,accnt_sender_3,accnt_sender_4,accnt_sender_5,accnt_sender_6,accnt_sender_7,accnt_sender_8,accnt_sender_9,...,accnt_sender_102,accnt_sender_103,accnt_sender_104,accnt_sender_105,accnt_sender_106,accnt_sender_107,accnt_sender_108,accnt_sender_109,accnt_sender_110,accnt_sender_111
0,10003741,5,3,2,1,0,0,0,0,0,...,0.00,0.00,0.00,0.0,0.0,0,0,0,0,0
1,10004362,17,9,6,3,1,2,0,0,0,...,0.00,0.00,0.00,0.0,0.0,0,0,0,0,0
2,1001738,8,7,6,5,2,0,0,0,0,...,0.00,0.00,0.00,0.0,0.0,0,0,0,0,0
3,10021750,19,5,4,3,0,0,0,0,0,...,0.00,0.00,0.00,0.0,0.0,0,0,0,0,0
4,10033036,4,3,3,1,0,1,1,1,0,...,12.02,12.02,12.02,0.0,0.0,4,4,4,0,0


Number of NaN values of each column

In [84]:
acct_sender.isnull().sum()[acct_sender.isnull().sum()>0]

Series([], dtype: int64)

There are no missing values in this dataset, so there is no further data processing required. We shall make a copy of this dataset and call it sender

In [85]:
sender=acct_sender


#### New Account Memo sent

In [86]:
acct_memo_sent=pd.read_csv("data/new_acct_memo_sent.csv", index_col=0)

In [87]:
print("The dimensions of the dataset are: ", acct_memo_sent.shape)

The dimensions of the dataset are:  (841975, 53)


In [88]:
acct_memo_sent.head()

,cust_id,new_acct_memo_sent_1,new_acct_memo_sent_2,new_acct_memo_sent_3,new_acct_memo_sent_4,new_acct_memo_sent_5,new_acct_memo_sent_6,new_acct_memo_sent_7,new_acct_memo_sent_8,new_acct_memo_sent_9,...,new_acct_memo_sent_43,new_acct_memo_sent_44,new_acct_memo_sent_45,new_acct_memo_sent_46,new_acct_memo_sent_47,new_acct_memo_sent_48,new_acct_memo_sent_49,new_acct_memo_sent_50,new_acct_memo_sent_51,new_acct_memo_sent_52
0,10001681,12,105.70,17.5,2.642500,0,0.0,0.0,0.0,3,...,400.0,14.436250,0,0.0,0.0,0.0,0,0.00,0.00,0.000000
1,10002304,2,61.00,55.0,1.452381,0,0.0,0.0,0.0,1,...,0.0,0.000000,0,0.0,0.0,0.0,2,47.00,27.00,1.119048
2,10002835,12,201.39,71.0,2.615455,0,0.0,0.0,0.0,0,...,72.0,0.935065,0,0.0,0.0,0.0,3,128.16,78.16,1.664416
3,10004172,1,6.00,6.0,0.750000,0,0.0,0.0,0.0,0,...,20.0,2.500000,0,0.0,0.0,0.0,0,0.00,0.00,0.000000
4,10004933,5,108.30,65.0,9.025000,0,0.0,0.0,0.0,0,...,0.0,0.000000,0,0.0,0.0,0.0,0,0.00,0.00,0.000000


Number of NaN values of each column

In [89]:
acct_memo_sent.isnull().sum()[acct_memo_sent.isnull().sum()>0]

Series([], dtype: int64)

There are no missing values in this dataset, so there is no further data processing required. We shall make a copy of this dataset and call it memo_sent

In [90]:
memo_sent=acct_memo_sent


#### New Account Memo received

In [91]:
acct_memo_received=pd.read_csv("data/new_acct_memo_rcv.csv", index_col=0)

In [92]:
print("The dimensions of the dataset are: ", acct_memo_received.shape)

The dimensions of the dataset are:  (771148, 53)


In [93]:
acct_memo_received.head()

,cust_id,new_acct_memo_rcv_1,new_acct_memo_rcv_2,new_acct_memo_rcv_3,new_acct_memo_rcv_4,new_acct_memo_rcv_5,new_acct_memo_rcv_6,new_acct_memo_rcv_7,new_acct_memo_rcv_8,new_acct_memo_rcv_9,...,new_acct_memo_rcv_43,new_acct_memo_rcv_44,new_acct_memo_rcv_45,new_acct_memo_rcv_46,new_acct_memo_rcv_47,new_acct_memo_rcv_48,new_acct_memo_rcv_49,new_acct_memo_rcv_50,new_acct_memo_rcv_51,new_acct_memo_rcv_52
0,10001395,1,5.0,5.0,0.238095,0,0.0,0.0,0.0,4,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.000000
1,10001440,1,25.0,25.0,0.961538,0,0.0,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.000000
2,10001463,0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.000000
3,10001681,2,17.0,9.0,1.307692,0,0.0,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,1,9.0,9.0,0.692308
4,10001834,0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.000000


Number of NaN values of each column

In [94]:
acct_memo_received.isnull().sum()[acct_memo_received.isnull().sum()>0]

Series([], dtype: int64)

There are no missing values in this dataset, so there is no further data processing required. We shall make a copy of this dataset and call it memo_rcv

In [95]:
memo_rcv=acct_memo_received

### Combining all the train data

In [96]:
base_train = pd.read_csv("data/train_df.csv")
print("Number of datapoints of class 0 = ",(base_train["target"] == 0).astype(int).sum())
print("Number of datapoints of class 1 = ",(base_train["target"] == 1).astype(int).sum())


Number of datapoints of class 0 =  640000
Number of datapoints of class 1 =  80000


In [97]:
base_test = pd.read_csv("data/test_cust_id.csv")

In [98]:
base_train.head()

,cust_id,target
0,46006086,0.0
1,13494269,0.0
2,58609039,0.0
3,14920847,0.0
4,59616399,1.0


It's important to keep in mind the class imbalance that we keep above in later analyses 

In [99]:
to_merge=[cash_fail, finan_instr,fraud,sender,receiver,memo_sent,memo_rcv]
total = reduce(lambda  left,right: pd.merge(left,right,on=['cust_id'], how='outer'), to_merge)

In [100]:
print("Dimensions of the combined dataframe are:", total.shape)

Dimensions of the combined dataframe are: (900000, 384)


In [101]:
total.isnull().sum()[total.isnull().sum()>0].to_csv("Missing_values.csv")

In [102]:
# sns.heatmap(total.isnull(), cbar=False)

* 128852 received 
* 330896 new acct fi
* 27110 fraud update feature
* 58025 sent
* 850022 payment cashout

##### Creating the training and test dataset

In [103]:
train=pd.merge(base_train,total, on=['cust_id'],how='inner')
train.shape

(720000, 385)

In [104]:
# sns.heatmap(train.isnull(), cbar=False)

In [105]:
test=pd.merge(base_test,total, on=['cust_id'],how='inner')
test.shape

(180000, 384)

In [106]:
# sns.heatmap(test.isnull(), cbar=False)

In [107]:
train.isnull().sum()[train.isnull().sum()>0].to_csv("Missing_values_train.csv")
test.isnull().sum()[test.isnull().sum()>0].to_csv("Missing_values_test.csv")

In [108]:
to_merge=[train,cash_fail, finan_instr,fraud,sender,receiver,memo_sent,memo_rcv]
intersection_train = reduce(lambda  left,right: pd.merge(left,right,on=['cust_id'], how='inner'), to_merge)
intersection_train.shape

(22848, 768)

In [109]:
# X=train.drop(columns=["target","cust_id"])
# y=train["target"]

#### Attempting to split the customers into New and Returning

In [110]:
print(total.isnull().sum()[total.isnull().sum()>0]) 

payment_cashout_fail_1    850022
payment_cashout_fail_2    850022
payment_cashout_fail_3    850022
payment_cashout_fail_4    850022
payment_cashout_fail_5    850022
                           ...  
new_acct_memo_rcv_48      128852
new_acct_memo_rcv_49      128852
new_acct_memo_rcv_50      128852
new_acct_memo_rcv_51      128852
new_acct_memo_rcv_52      128852
Length: 383, dtype: int64


After looking at the null value heatmap, we decided to split the dataframe into new and returning customers according to missing values of the new_acct_fi dataframe
We also chose to drop all the payment cashout fail columns. Simply because too many of the values were missing

In [111]:
train.drop(columns=cashout_fail_cols,inplace=True)

In [112]:
test.drop(columns=cashout_fail_cols,inplace=True)

In [113]:
train.head()

,cust_id,target,num,avg,new_acct_fi_1,new_acct_fi_2,new_acct_fi_3,new_acct_fi_4,new_acct_fi_5,new_acct_fi_6,...,new_acct_memo_rcv_43,new_acct_memo_rcv_44,new_acct_memo_rcv_45,new_acct_memo_rcv_46,new_acct_memo_rcv_47,new_acct_memo_rcv_48,new_acct_memo_rcv_49,new_acct_memo_rcv_50,new_acct_memo_rcv_51,new_acct_memo_rcv_52
0,46006086,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,13494269,0.0,20.0,674843.546,1612.046030,1612.046030,1612.046030,1613.041667,1613.041667,1613.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0,100.0,4.166667
2,58609039,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,14920847,0.0,NaN,NaN,1565.431042,1565.431042,1565.431042,1566.041667,1566.041667,1566.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,59616399,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
train_old=train[train["new_acct_fi_1"].notna()]
train_new=train[pd.isna(train["new_acct_fi_1"])]
train_new.drop(columns=acct_fi_cols,inplace=True)
X_train_old=train_old.drop(columns=["target","cust_id"])
y_train_old=train_old["target"]
X_train_new=train_new.drop(columns=["target","cust_id"])
y_train_new=train_new["target"]

/home/gurmehar/anaconda3/envs/insti/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [131]:
test_old=test[test["new_acct_fi_1"].notna()]
test_new=test[pd.isna(test["new_acct_fi_1"])]
test_new.drop(columns=acct_fi_cols,inplace=True)

/home/gurmehar/anaconda3/envs/insti/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


#### Processing missing values

In [132]:
X_train_old.fillna(0,inplace=True)
X_train_new.fillna(0,inplace=True)
y_train_old.fillna(0,inplace=True)
y_train_new.fillna(0,inplace=True)

/home/gurmehar/anaconda3/envs/insti/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


### Random Forest on the whole dataset as a benchmark

In [133]:
# X.dropna(inplace=True)

# RF=RandomForestClassifier(verbose=2)

# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)
# RF.fit(X_train,y_train)
# y_pred=RF.predict(X_test)
# print("Score: ", roc_auc_score(y_pred,y_test))


In [134]:
temp_test=test.copy()

In [135]:
# test=test.fillna(0)
# output=RF.predict(test.drop(columns=["cust_id"]))


### Random forest 

In [136]:
RF_new=RandomForestClassifier(verbose=2)
RF_old=RandomForestClassifier(verbose=2)

#### New customers

In [137]:
X_RF_new_train, X_RF_new_test, y_RF_new_train, y_RF_new_test = train_test_split(X_train_new,y_train_new,test_size=0.25,random_state=42)
RF_new.fit(X_RF_new_train,y_RF_new_train)
y_pred_RF_new=RF_new.predict(X_RF_new_test)
print("Score: ", roc_auc_score(y_pred_RF_new,y_RF_new_test))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Score:  0.6970605837776008


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.9s finished


#### Old customers

In [138]:
X_RF_old_train, X_RF_old_test, y_RF_old_train, y_RF_old_test = train_test_split(X_train_old,y_train_old,test_size=0.25,random_state=42)
RF_old.fit(X_RF_old_train,y_RF_old_train)
y_pred_RF_old=RF_old.predict(X_RF_old_test)
print("Score: ", roc_auc_score(y_pred_RF_old,y_RF_old_test))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  5.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Score:  0.721957071706723


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.3s finished


#### Computing the score for the combined model

In [139]:
y_pred_RF_train= np.hstack([y_pred_RF_old,y_pred_RF_new])
y_RF_test_train= np.hstack([y_RF_old_test,y_RF_new_test])
# print("Score: ", roc_auc_score(y_pred_RF_new,y_RF_new_test))
# print("Score: ", roc_auc_score(y_pred_RF_old,y_RF_old_test))
print(" Overall Score: ", roc_auc_score(y_pred_RF_train,y_RF_test_train))


 Overall Score:  0.7159668090073927


### Making submissions

In [140]:
output_new=output_new # Replace with output DF of choice
output_old=output_old
output=pd.concat([output_old,output_new],axis=0)
IDs=pd.concat([test_old["cust_id"],test_new["cust_id"]],axis=0)
submission_array=np.array([output]).T
submission_df=pd.DataFrame(submission_array)
final_submission=pd.concat([IDs,submission_df],axis=1)

NameError: name 'output_new' is not defined

In [ ]:
final_submission.columns=["cust_id","target"]
final_submission=final_submission.set_index('cust_id')
final_submission.to_csv("Random_forest_base.csv")